<a href="https://colab.research.google.com/github/harvard-visionlab/sroh/blob/main/2022/simple_actxgrad_importance_of_features_wrt_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

## data loader

We want to pass imagenet data through the model, Im not sure how you access that data, but you need the official Imagenet dset on your file system, with 'train' & 'val' subfolders, with category-wise subfolders 'n01440764', 'n01443537' etc. point the following 'data_folder' variable to that imagenet path.

In [5]:
#downloading validation dataset
!wget -c https://www.dropbox.com/s/6vu07wtshpqpcr2/val.tar.gz
!tar -xf val.tar.gz
!rm val.tar.gz

--2022-08-12 14:11:08--  https://www.dropbox.com/s/6vu07wtshpqpcr2/val.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6vu07wtshpqpcr2/val.tar.gz [following]
--2022-08-12 14:11:08--  https://www.dropbox.com/s/raw/6vu07wtshpqpcr2/val.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc66010023f1c6aed7e237c3387e.dl.dropboxusercontent.com/cd/0/inline/Bq0fld2m3c-SHo0FDlF6jveXumjxkW0Qy2Twz_woQD1PjtdyV_LZfKZFma2eV8VGJ8PhiRqMwiM6M47OwzyeyFrSqAtKd4CDdw7EVyBBqFkpI_CLqo2iI3B-M40n-XVaAJWfTRmKBpqvHDxn51Ubeu3JbcDqH0wAMyP2wpQpHM9Quw/file# [following]
--2022-08-12 14:11:09--  https://uc66010023f1c6aed7e237c3387e.dl.dropboxusercontent.com/cd/0/inline/Bq0fld2m3c-SHo0FDlF6jveXumjxkW0Qy2Twz_woQD1PjtdyV_LZfKZFma2eV8VGJ8PhiRqMwi

In [6]:
data_folder = '/content/val'

In [10]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

kwargs = {'num_workers': 4, 'pin_memory': True, 'sampler':None} if 'cuda' in device else {}

dset = ImageFolder(data_folder,transform=transform)
print(dset)
assert len(dset) == 50000, f"Oops, expected 50000 images, got {len(dset)}"

dloader = DataLoader(dset,
                     batch_size=256,
                     shuffle=False,
                     **kwargs)


Dataset ImageFolder
    Number of datapoints: 50000
    Root location: /content/val
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


### Prepping model

We want to 'score' the features in a layer by how much they might affect the model loss. We can approximate that as the average activationxgrad passing through a feature. Intuitively, this works because the gradient measures how much 'changing' the feature would affect the loss, while the activation size measures how much the feature would change (setting a high activation feature to 0 activation is a big change). Well use a 'hook' to save activation/gradient values.

In [32]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, transform = torch.hub.load("harvard-visionlab/open_ipcl", "alexnetgn_supervised_ref13_augset1_1x")
model = model.to(device)

Using cache found in /root/.cache/torch/hub/harvard-visionlab_open_ipcl_main


In [33]:
#little trick for getting a dictionary with reference names for each module in your model, at all nestings
layers = dict([*model.named_modules()])
#so now we can reference modules with a string;
print(layers.keys())
layers['fc6.0']

dict_keys(['', 'conv_block_1', 'conv_block_1.0', 'conv_block_1.1', 'conv_block_1.2', 'conv_block_1.3', 'conv_block_2', 'conv_block_2.0', 'conv_block_2.1', 'conv_block_2.2', 'conv_block_2.3', 'conv_block_3', 'conv_block_3.0', 'conv_block_3.1', 'conv_block_3.2', 'conv_block_4', 'conv_block_4.0', 'conv_block_4.1', 'conv_block_4.2', 'conv_block_5', 'conv_block_5.0', 'conv_block_5.1', 'conv_block_5.2', 'conv_block_5.3', 'ave_pool', 'fc6', 'fc6.0', 'fc6.1', 'fc6.2', 'fc7', 'fc7.0', 'fc7.1', 'fc7.2', 'fc8', 'fc8.0'])


Linear(in_features=9216, out_features=4096, bias=True)

In [34]:
from torch import nn, Tensor
from typing import Dict, Iterable, Callable

class actgrad_extractor(nn.Module):
    def __init__(self, model: nn.Module, layers: Iterable[str]):
        super().__init__()
        self.model = model
        self.layers = layers
        self.activations = {layer: None for layer in layers}
        self.gradients = {layer: None for layer in layers}
        self.hooks = {'forward':{},
                      'backward':{}}   #saving hooks to variables lets us remove them later if we want
        
        for layer_id in layers:
            layer = dict([*self.model.named_modules()])[layer_id]
            self.hooks['forward'][layer_id] = layer.register_forward_hook(self.save_activations(layer_id)) #execute on forward pass
            self.hooks['backward'][layer_id] = layer.register_backward_hook(self.save_gradients(layer_id))    #execute on backwards pass

    def save_activations(self, layer_id: str) -> Callable:
        def fn(module, input, output):  #register_hook expects to recieve a function with arguments like this
            #output is what is return by the layer with dim (batch_dim x out_dim), sum across the batch dim
            batch_summed_output = torch.sum(torch.abs(output),dim=0).detach().cpu()
            if self.activations[layer_id] is None:
                self.activations[layer_id] = batch_summed_output
            else:
                self.activations[layer_id] +=  batch_summed_output
        return fn
    
    def save_gradients(self, layer_id: str) -> Callable:
        def fn(module, grad_input, grad_output):
            batch_summed_output = torch.sum(torch.abs(grad_output[0]),dim=0).detach().cpu() #grad_output is a tuple with 'device' as second item
            if self.gradients[layer_id] is None:
                self.gradients[layer_id] = batch_summed_output
            else:
                self.gradients[layer_id] +=  batch_summed_output 
        return fn
    
    def remove_all_hooks(self):      
        for hook in self.hooks['forward'].values():
            hook.remove()
        for hook in self.hooks['backward'].values():
            hook.remove()
    

### Running model

In [35]:
target_layers = ["conv_block_3.0","fc6.0"]

model_actgrad_extractor = actgrad_extractor(model, layers=target_layers)
criterion = nn.CrossEntropyLoss()

In [36]:
from fastprogress import progress_bar 

iter_dataloader = iter(dloader)
iters = len(iter_dataloader)  #if you want to test this out quickly just set this to a small number
print('total batches: ' + str(iters)) 
for it in progress_bar(range(iters)):
    inputs, target = next(iter_dataloader)
    inputs = inputs.to(device)
    target = target.to(device)

    model.zero_grad()
    
    output = model(inputs)
    
    loss = criterion(output,target)
    loss.backward()
    
model_actgrad_extractor.remove_all_hooks()

total batches: 196


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1053: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [37]:
#get average by dividing result by length of dset
activations = model_actgrad_extractor.activations
gradients = model_actgrad_extractor.gradients

for l in target_layers:
    activations[l] /= len(dset)
    gradients[l] /= len(dset)

In [40]:
#scores are just actxgrad! Do with them what you want

scores = {}
for l in target_layers:
    scores[l] = activations[l]*gradients[l]
    print(l, scores[l].shape)

conv_block_3.0 torch.Size([384, 13, 13])
fc6.0 torch.Size([4096])
